In [608]:
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn import preprocessing

pd.set_option("display.max_columns", 100)

DATA_PATH = Path.cwd().parent / "ML project" 



from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer


from sklearn.multioutput import MultiOutputClassifier

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score

RANDOM_SEED = 6    # Set a random seed for reproducibility!

In [609]:
%matplotlib inline
import matplotlib.pyplot as plt


In [610]:
def formatdata(dataFrame):
    dataFrame['pickup_time'] = dataFrame['pickup_time']+ '.099'
    dataFrame['pickup_time'] = dataFrame['pickup_time'].apply(lambda x:  x.replace("/","-"))
    dataFrame['drop_time'] = dataFrame['drop_time']+ '.099'


    #features_df['pickup_time'].values.replace('/','-',regex=True)

    pd.to_datetime(dataFrame['pickup_time'], format='%m-%d-%Y %H:%M:%S.%f', errors='ignore')
    pd.to_datetime(dataFrame['drop_time'], format='%m/%d/%Y %H:%M:%S.%f', errors='ignore')

    #features_df['ts'] = pd.to_timedelta(features_df['pickup_time'], unit='ns').features_df.total_seconds().astype(int)
    #features_df['pickup_timez'] = pd.Timestamp(features_df['pickup_time'])
    dataFrame['pickup_time'] = dataFrame['pickup_time'].apply(lambda x: datetime.timestamp(datetime.strptime(x,'%m-%d-%Y %H:%M.%f')))
    dataFrame['drop_time'] = dataFrame['drop_time'].apply(lambda x: datetime.timestamp(datetime.strptime(x,'%m/%d/%Y %H:%M.%f')))

    dataFrame['timediff'] = dataFrame['drop_time'] - dataFrame['pickup_time']
    dataFrame['distance'] = ((dataFrame['drop_lat']-dataFrame['pick_lat'])**2+(dataFrame['drop_lon']-dataFrame['pick_lon'])**2)**0.5
    

    return dataFrame

    cols_to_norm = ['duration','meter_waiting',
                    'meter_waiting_fare','meter_waiting_till_pickup',
                    'pick_lat','pick_lon','drop_lat','drop_lon','fare','timediff','distance']
    n_test = dataFrame[cols_to_norm]



    x = n_test.values

    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    normalized_df=(x-x.mean())/x.std()
    n_test = pd.DataFrame(normalized_df, columns=cols_to_norm,index=dataFrame.index)

    l_test = dataFrame.drop(cols_to_norm, axis=1)
    train = pd.concat([n_test, l_test], axis=1)
    return train

In [611]:
train_df = pd.read_csv(
    DATA_PATH / "train.csv", 
    index_col="tripid"
)



train_df['label'] = train_df['label'].map({"correct":1, "incorrect":0})
train_df_cols = train_df.columns.values
li = list(train_df_cols)
features_df = train_df[li[0:-1]]
labels_df = train_df[li[-1]]

features_df = formatdata(features_df)




numeric_cols = features_df.columns[features_df.dtypes != object].values
categorical_features = features_df.columns[features_df.dtypes == object].values



def removeColumns(npArry, columnNames):
    li = list(npArry)
    for i in columnNames:
        li.remove(i)
    return np.array(li)


def powerAll(data_frame, exceptList,power):
    allColumns = data_frame.columns.values
    for i in allColumns:
        if(i not in exceptList):
            data_frame[i] = data_frame[i]**power
 

exceptList = features_df.columns[features_df.dtypes == "object"].values

#numeric_cols = removeColumns(numeric_cols,['drop_time', 'pickup_time'])  

powerAll(features_df, exceptList,1)
#exceptList = features_df.columns[features_df.dtypes == "object"].values


numeric_preprocessing_steps = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('simple_imputer', SimpleImputer(strategy='median'))
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# create the preprocessor stage of final pipeline
# each entry in the transformer list is a tuple of
# (name you choose, sklearn transformer, list of columns)
preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, numeric_cols),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder = "drop"
)

C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [612]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor
#estimators = MultiOutputClassifier(
 #   estimator=MLPClassifier(solver='lbfgs', alpha=.55, hidden_layer_sizes=(4,2 ), random_state=1, max_iter=10000000)
#)
#estimators = MultiOutputClassifier(
 #   estimator=SVC(gamma='auto', probability=True),
#)
#estimators =SVC(gamma='auto')
#estimators=LogisticRegression(penalty="l2", C=1,solver='lbfgs',max_iter=1000000)
estimators=MLPClassifier(solver='lbfgs', alpha=.45, hidden_layer_sizes=(7,2 ), random_state=1, max_iter=10000000)

In [613]:
full_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", estimators),
])

X_train, X_eval, y_train, y_eval = train_test_split(
    features_df,
    labels_df,
    test_size=0.33,
    shuffle=True,
    stratify=labels_df,
    random_state=6
)


In [614]:
%%time

# Train model
full_pipeline.fit(X_train, y_train)


# Predict on evaluation set
# This competition wants probabilities, not labels
preds = full_pipeline.predict(X_eval)
preds



Wall time: 17 s


array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [615]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
y_preds = pd.DataFrame(
    preds,
    index = y_eval.index
)
print("y_preds.shape:", y_preds.shape)


y_preds.shape: (5669, 1)


In [616]:
accuracy_score(y_eval, y_preds)

0.9481390015875816

In [617]:
confusion_matrix(y_eval, y_preds)

array([[ 340,  215],
       [  79, 5035]], dtype=int64)

In [618]:
from sklearn.metrics import recall_score
recall= recall_score(y_eval, y_preds)

In [619]:
from sklearn.metrics import precision_score
precission = precision_score(y_eval, y_preds)

In [620]:
from sklearn.metrics import f1_score
f1 = f1_score(y_eval, y_preds)
print(recall)
print(precission)
print(f1)

0.9845522096206492
0.959047619047619
0.9716325742956388


In [621]:
%%time 

full_pipeline.fit(features_df, labels_df)

None

Wall time: 18.9 s


In [622]:
test_features_df = pd.read_csv(DATA_PATH / "test.csv", 
                               index_col="tripid")

test_features_df = formatdata(test_features_df)
powerAll(test_features_df, exceptList,1)
#exceptList = features_df.columns[features_df.dtypes == "object"].values
#test_features_df = formatdata(test_features_df)





In [623]:
test_probas = full_pipeline.predict(test_features_df)

test_probas

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [624]:
test_preds = pd.DataFrame(
    {'prediction':test_probas},
    index = test_features_df.index
)

print(test_preds['prediction'].max())
print(test_preds['prediction'].min())

1
0


In [625]:
test_preds.to_csv(DATA_PATH / 'my_submission.csv', index=True)

In [626]:
submission_df.head()

NameError: name 'submission_df' is not defined

In [ ]:
# Make sure we have the rows in the same order
np.testing.assert_array_equal(test_features_df.index.values, 
                              submission_df.index.values)

# Save predictions to submission data frame
submission_df["h1n1_vaccine"] = test_preds[1]

submission_df.head()

In [ ]:
submission_df.to_csv(DATA_PATH / 'my_submission.csv', index=True)

In [ ]:
from datetime import datetime
to = '11/01/2019  00:20:00'
to += '.099'
froms = '11/01/2019  00:34:00'+'.099'
dateObjectto = datetime.strptime(to, '%m/%d/%Y %H:%M:%S.%f')
dateObjectfrom = datetime.strptime(froms, '%m/%d/%Y %H:%M:%S.%f')
print(datetime.timestamp(dateObjectfrom) - datetime.timestamp(dateObjectto))


In [ ]:
train_df = pd.read_csv(
    DATA_PATH / "train.csv", 
    index_col="tripid"
)
train_df['label'] = train_df['label'].map({"correct":1, "incorrect":0})
train_df_cols = train_df.columns.values
li = list(train_df_cols)
features_df = train_df[li[0:-1]]
labels_df = train_df[li[-1]]

In [ ]:
features_df.describe()

In [ ]:
print(features_df.dtypes)

In [ ]:
import matplotlib.pyplot as plt

features_df[features_df.dtypes[(features_df.dtypes=="float64")]
                        .index.values].hist(figsize=[11,11],
                                            range=[features_df['drop_lat'].min(), features_df['drop_lat'].max()])


In [ ]:
from sklearn import preprocessing

cols_to_norm = ['duration','meter_waiting',
                'meter_waiting_fare','meter_waiting_till_pickup',
                'pick_lat','pick_lon','drop_lat','drop_lon','fare']
n_test = features_df[cols_to_norm]


x = n_test.values

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

n_test = pd.DataFrame(x_scaled, columns=cols_to_norm,index=features_df.index)

l_test = features_df.drop(cols_to_norm, axis=1)
train = pd.concat([n_test, l_test], axis=1)
train.columns

In [ ]:
import matplotlib.pyplot as plt

train[train.dtypes[(train.dtypes=="float64")]
                        .index.values].hist(figsize=[11,11],range=[0,1])
